In [ ]:
#| export
from __future__ import annotations

In [ ]:
#| default_exp base_widget

In [ ]:
# %reload_ext autoreload
# %autoreload 0

# Base Widget
> Base class of custom jupyter widgets

## Prologue

In [ ]:
#| export
import asyncio
import inspect
import pathlib
from functools import partial
from functools import wraps
from typing import Any
from typing import Callable
from typing import cast
from typing import Protocol
from typing import Type
from typing_extensions import Self

import anywidget
import fastcore.all as F
import ipywidgets as W
import traitlets as T
from IPython.display import display

import vwidget
from vutil.imports import AD


In [ ]:
import datetime as dt
import time
import uuid

from fastcore.test import *  # type: ignore [reportWildcardImportFromLibrary]
import pandas as pd
from vutil.test import test_is_not
from vutil.test import test_raises


 ----

In [ ]:
# if F.IN_IPYTHON:
#     import nest_asyncio
#     nest_asyncio.apply()

In [ ]:
#| export
if F.IN_NOTEBOOK:
    _BUNDLER_PATH = pathlib.Path("src")
else:
    _BUNDLER_PATH = pathlib.Path(inspect.getfile(vwidget)).parent / ("static")


ipywidgets/anywidgets deprecation warning

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


 ----

In [ ]:
# %gui asyncio

## Reflect widget

In [ ]:
#| exporti
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future


In [ ]:
#| exporti
def yield_for_change(widget, attribute):
    """Pause a generator to wait for a widget change event.
        
    This is a decorator for a generator function which pauses the generator on yield
    until the given widget attribute changes. The new value of the attribute is
    sent to the generator and is the value of the yield.
    """
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change.new)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.observe(next_i, attribute)
            # start the generator
            next(i)
        return inner
    return f


In [ ]:
#| export
class ReflectWidget(anywidget.AnyWidget):
    _id = 0
    _esm = _BUNDLER_PATH / 'reflect.js'
    _css = """
          .reflect-div {
            width: 40ch;
            height: 20px;
            background-image: linear-gradient(to right, #a1c4fd, #c2e9fb);
            # background-image: transparent;
            border: 0;
            # border-radius: 10px;
            # padding: 10px 50px;
            color: white;
          }
          """
    _rid = T.Unicode('').tag(sync=True)  # type: ignore
    ui = T.Bool(False).tag(sync=True)  # type: ignore
    # value = traitlets.Int(0).tag(sync=True)  # type: ignore
    # _html = traitlets.Unicode('').tag(sync=True)  # type: ignore
    query = T.Dict({}).tag(sync=True)

    @property
    def result(self) -> Any | None:
        return self.query['result'] if 'result' in self.query else None  # type: ignore
    
    def get_result(self, query, done: Callable[[Self], None] | None = None) -> asyncio.Future:
        async def f():
          x = await wait_for_change(self, 'query')
          if done:
            loop = asyncio.get_event_loop()
            loop.call_later(0.0, done, self)
        
        self.query = query
        return asyncio.ensure_future(f())
    
    def __init__(self, ui: bool = False, query: dict | None = None, *args, **kwargs):
        if ui: 
          self.ui = ui
        ReflectWidget._id += 1
        self._rid = f"__vutil-reflect-{self._id}"
        super().__init__(*args, **kwargs)
        if query is None:
          return
        self.query = query
        async def f():
          x = await wait_for_change(self, 'query')
        asyncio.ensure_future(f())


In [ ]:
display(w := ReflectWidget())
test_eq(w.query, {})


ReflectWidget()

In [ ]:
query = {'method':'getBoundingClientRect'}

def _f(w):
    # print(f"{w.query = }")
    test_ne(w.query, query)
    test_eq(w.result['height'], 0)
fut = w.get_result(query, _f)
test_is(fut.done(), False)


In [ ]:
w = ReflectWidget()
display(w)
query = {'kind':'html'}
def _f2(w):
    # print(f"{w.query = }")
    test_eq(w.result, 
            f'<div class="reflect-div" id="__vutil-reflect-{w._id}" style="display: none;">'
            f'reflect __vutil-reflect-{w._id}</div>')
fut = w.get_result(query, _f2)
test_is(fut.done(), False)


ReflectWidget()

Reflection is async. We must wait for the front end to send the widget state before we can reflect it.

In [ ]:
w = ReflectWidget(ui=True)
display(w)
fut = w.get_result(
    {'method':'getBoundingClientRect'},
    lambda x: test_eq(cast(dict, x.result)['height'], 20)
)
test_is(fut.done(), False)


ReflectWidget(ui=True)

Exception in callback _f2(ReflectWidget...ect-2</div>'})) at /var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_92540/3667747487.py:4
handle: <TimerHandle when=94222.0914365 _f2(ReflectWidget...ect-2</div>'})) at /var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_92540/3667747487.py:4>
Traceback (most recent call last):
  File "/Users/vic/dev/repo/project/vwidget/.micromamba/envs/vwidget/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_92540/3667747487.py", line 6, in _f2
    test_eq(w.result,
  File "/Users/vic/dev/repo/project/vwidget/.micromamba/envs/vwidget/lib/python3.10/site-packages/fastcore/test.py", line 37, in test_eq
    test(a,b,equals, cname='==')
  File "/Users/vic/dev/repo/project/vwidget/.micromamba/envs/vwidget/lib/python3.10/site-packages/fastcore/test.py", line 27, in test
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"
AssertionError: ==

Use `wait_for_change` or `yield_for_change` to wait for reflection to complete

In [ ]:
w = ReflectWidget()
display(w)


ReflectWidget()

In [ ]:
w.ui = True
w.query = {'method':'getBoundingClientRect'}

async def f1(w):
    x = await wait_for_change(w, 'query')
    # print(x)
    test_eq(x, w.query)
    if 'result' in  x:
        test_eq(cast(dict, w.result)['height'], 20)
        # print(f"{cast(dict, w.query)['result']['height'] = }")
asyncio.ensure_future(f1(w))
test_is(w.result, None)


In [ ]:
w2 = ReflectWidget()
display(w2)

w2.ui = True
w2.query = {'method':'getBoundingClientRect'}

@yield_for_change(w2, 'query')
def f2():
    x = yield
    # print(x)
    test_eq(x, w2.query)
    test_eq(cast(dict, w2.result)['height'], 20)
    # print(f"{cast(dict, w2.query)['result']['height'] = }")
f2()  # yield_for_change(w, 'query')(f)()
test_is(w2.result, None)


ReflectWidget()

## Themer widget

In [ ]:
#| export
class ThemerWidget(anywidget.AnyWidget):
    _esm = """
    export function render(view) {
        let div = document.getElementById("__vutil_themer");
        if (!div) {
          div = document.createElement("div");
          div.id = "__vutil_themer";
          div.innerHTML = `themer div`;
          div.style.display = "none";
          view.el.appendChild(div);
        }
        view.model.on("change:value", () => {
          let div = document.getElementById("__vutil_themer");
          div.style.display = view.model.get("ui") ? "block" : "none";
        });
    }
    """

    _css = """
    .cell-output-ipywidget-background {
        background-color: var(--vscode-editor-background) !important;
        # border: 1px solid var(--vscode-editor-background) !important;
        border: 1px solid rgb(53 154 92 / 35%);
    }
    #__vutil_themer {
      height: 20px;
      width: 100px;
      background-color: red;
    }
    """
    ui = T.Bool(False).tag(sync=True)  # type: ignore

class WidgetCSS(anywidget.AnyWidget):
    _esm = None
    def __init__(self, css: str):
        self._css = css
        super().__init__()


In [ ]:
w = W.Box([(r := ReflectWidget())], layout=W.Layout(height="80px"))
w

Box(children=(ReflectWidget(),), layout=Layout(height='80px'))

{'selector': 'div[class="cell-output-ipywidget-background"]', 'kind': 'style', 'prop': 'background-color', 'rid': '__vutil-reflect-6', 'result': 'rgb(255, 255, 255)'}


Exception in callback check_result.<locals>.f(ReflectWidget...31, 31, 31)'})) at /var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_92540/2495405475.py:2
handle: <TimerHandle when=94195.985518625 check_result.<locals>.f(ReflectWidget...31, 31, 31)'})) at /var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_92540/2495405475.py:2>
Traceback (most recent call last):
  File "/Users/vic/dev/repo/project/vwidget/.micromamba/envs/vwidget/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_92540/2495405475.py", line 5, in f
    test_eq(w.result, expected)
  File "/Users/vic/dev/repo/project/vwidget/.micromamba/envs/vwidget/lib/python3.10/site-packages/fastcore/test.py", line 37, in test_eq
    test(a,b,equals, cname='==')
  File "/Users/vic/dev/repo/project/vwidget/.micromamba/envs/vwidget/lib/python3.10/site-packages/fastcore/test.py", line 27, in test
    assert

{'selector': 'div[class="cell-output-ipywidget-background"]', 'kind': 'style', 'prop': 'background-color', 'rid': '__vutil-reflect-6', 'result': 'rgb(31, 31, 31)'}


In [ ]:
def check_result(expected, show_query=False):
    def f(w):
        if show_query:
            print(w.query)
        test_eq(w.result, expected)
    return f


In [ ]:
fut = r.get_result(
    {'selector':'div[class="cell-output-ipywidget-background"]', 'kind':'style', 'prop':'background-color'}, 
    check_result('rgb(255, 255, 255)', True))
test_is(fut.done(), False)
fut


<Task pending name='Task-8' coro=<ReflectWidget.get_result.<locals>.f() running at /var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_92540/2668587759.py:28>>

In [ ]:
tw = ThemerWidget()
tw

ThemerWidget()

In [ ]:
fut = r.get_result(
    {'selector':'div[class="cell-output-ipywidget-background"]', 'kind':'style', 'prop':'background-color'}, 
    check_result('rgb(30, 30, 30)', True))
test_is(fut.done(), False)
fut


<Task pending name='Task-9' coro=<ReflectWidget.get_result.<locals>.f() running at /var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_92540/2668587759.py:28>>

## BaseExplorerApp
> Simple `Box` subclass for composing widgets

In [ ]:
def create_expanded_button(description, button_style):
    return W.Button(description=description, button_style=button_style, layout=W.Layout(height='auto', width='auto'))    
    # return W.Button(description=description, button_style=button_style, layout=W.Layout(height='100%', width='100%'))

In [ ]:
header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

In [ ]:
app = W.AppLayout(
    # header=None,#header_button,
    left_sidebar=left_button,
    center=center_button,#W.Box([center_button]),
    # right_sidebar=None,#right_button,
    # footer=None,#footer_button,
    pane_widths=[2, 4, 3],
    pane_heights=[1, 5, 1],
    # height='220px',  # these two are forwarded to `layout`
    # width='100%',
    layout=dict(
      height='220px',
      max_height='320px',
      width='100%',)
  )
app
# box = W.Box([app])
# box


AppLayout(children=(Button(button_style='info', description='Left', layout=Layout(grid_area='left-sidebar', he…

In [ ]:
#| export
class DebugWidgetP(Protocol):
    def clear(self, *args): ...
    def log(self, msg): ...
    def log_inc(self, msg=''): ...

class _NoopDebugWidget:
    def clear(self, *args): ...
    def log(self, msg): ...
    def log_inc(self, msg=''): ...
NOOPDCONS = _NoopDebugWidget()

class BaseExplorerApp(W.AppLayout):
    "Base class for ipywidgets composite to explore data"
    _w: AD  # widget subcomponents
    _css = ''
    _exclude = ()
    value = T.Dict({}, read_only=True)
    dcons: DebugWidgetP
    themer: WidgetCSS
    reflect: ReflectWidget
    
    @classmethod
    def _setup_css(cls) -> WidgetCSS | None:
        if not cls._css:
            return None
        if not hasattr(cls, '_css_cls'):
            n, ll = f"{cls.__name__}_CSS", {}
            exec(f"class {n}(WidgetCSS): ...", globals(), ll)
            cls._css_cls:Type[WidgetCSS] = ll[n]
        return cls._css_cls(cls._css)
    def _setup_dcons(self, dcons: bool | DebugWidgetP | None = None) -> DebugWidgetP: ...

    def d(self): pass

    # ---------- ---------- Model ---------- ----------
    def setup_model(self, *args) -> dict[str, Any]:
        """setup model and initial state.
        For our purposes, the model is just a mapping of widget names to actual widget values
        """
        return AD()

    # ---------- ---------- UI view ---------- ----------
    @property
    def widgets(self) -> dict[str, W.DOMWidget]:
        return AD(self._w)
    def _update_ui(self): pass
    def setup_widgets(self, widgets: dict[str, W.DOMWidget] = {}) -> dict[str, W.DOMWidget]:
        return AD(widgets)
    def setup_layout(self):
        # ww = []
        # ensure themer and/or reflect is first
        # try:
        #     ww.append(self._w.themer)
        # except AttributeError:
        #     pass
        # try:
        #     ww.append(self._w.reflect)
        # except AttributeError:
        #     pass
        # # # note widgets are stacked in self._w insertion order
        # # ww.extend([w for k, w in self._w.items() if k not in self._exclude])
        # # base_box = W.Box(
        # #     ww, 
        # #     # layout=W.Layout(
        # #     #     # display='flex',
        # #     #     # flex_flow='row',
        # #     #     # justify_content='flex-start',
        # #     #     # height=f"{height}px",
        # #     #     # border='1px solid red',
        # #     # )
        # # )
        # # return base_box
        # return ww
        if self._w:
            return {'center': W.Box(list(self._w.values()), layout=W.Layout(width='100%', height='100%'))}
        return {}
    
    # ---------- ---------- UX ---------- ----------
    def _setup_state(self, state: dict[str, Any], **kwargs):
        "setup initial values of widgets"
        for k in tuple(kwargs.keys()):
            if k in self._w and k not in self._exclude:
                v = kwargs.pop(k)
                # self._w[k].value = v
                state[k] = v
        # state.update({
        #     k:w.value for k, w in self._w.items() 
        #     if k not in self._exclude and hasattr(w, 'value')})
        return kwargs
    def on_value_change(self, change):
        # print(f"{type(self).__name__}.on_value_change - {change['new']}")
        pass
    def _on_widget_value_change(self, widget_name, change):
        # self.value[widget_name] = change['new']
        # tt = time.perf_counter()
        self.set_trait('value', self.value | {widget_name:change['new']})
        # print(f"{tt} - {type(self).__name__}._on_widget_value_change - {widget_name}: {change['new']}")
    def setup_ux(self, state):
        "state contains initial values of widgets from `setup_model` and defaults from `_defs`"
        # setup initial values of widgets
        for k, v in state.items():
            self._w[k].value = v
        # setup initial app value
        self.set_trait('value', state)
        # setup widget callbacks
        for k,w in self._w.items():
            if k not in self._exclude and hasattr(w, 'value'):
                self._w[k].observe(partial(self._on_widget_value_change, k), names='value')
        self.observe(self.on_value_change, names='value')  # type: ignore
    # ---------- ---------- initialization ---------- ----------
    def show(self):
        ww = []
        try:
            ww.append(self.themer)
        except AttributeError:
            pass
        try:
            ww.append(self.reflect)
        except AttributeError:
            pass
        ww.append(self)
        d = self.dcons
        # if d is not None and d is not NOOPDCONS:
        #     ww.append(cast(Any, d))
        display(W.Box(ww), d) if d is not None and d is not NOOPDCONS else display(W.Box(ww))
    def __init__(self, 
            *model,  # model initial state, forwarded to _setup_model
            show: bool = True, 
            dcons: bool | DebugWidgetP | None = None, 
            reflect: bool = True,
            **kwargs  # widgets items values, forwarded to _setup_values
    ):
        self.dcons = self._setup_dcons(dcons)
        if reflect:
            self.reflect = ReflectWidget()
        if (_ := self._setup_css()):
            self.themer = _
        # ---------- model
        state = self.setup_model(*model)
        # --------- view
        self._w = cast(AD, self.setup_widgets(kwargs.pop('widgets', {})))
        panes = self.setup_layout()
        # ----- defaults
        defs = getattr(type(self), '_defs', None)
        if defs:
            defs.update(kwargs)
            kwargs = defs
        # ----- state
        # self._setup_values(**kwargs)
        kwargs = self._setup_state(state, **kwargs)  # after this, kwargs should only have AppLayout props
        super().__init__(**panes, width=kwargs.pop('width', '100%'), **kwargs)
        # ---------- UX
        if show:
            self.show()
        self.setup_ux(state)
        self.d()



In [ ]:
w = BaseExplorerApp()
test_is_not(w.reflect, None)
test_is(cast(ReflectWidget, w.reflect).result, None)
test_eq((_ for _ in w.keys if _[0] != '_'), [  # type: ignore
    'box_style',
    'children',
    'layout',
    'tabbable',
    'tooltip'])
test_eq(w.value, {})

Box(children=(ReflectWidget(), BaseExplorerApp(layout=Layout(width='100%'))))

{'x': 53, 'y': 16098, 'width': 878, 'height': 0, 'top': 16098, 'right': 931, 'bottom': 16098, 'left': 53}


In [ ]:
assert w.reflect is not None
w.reflect.get_result(
    {'selector':'.widget-box', 'method':'getBoundingClientRect'}, 
    lambda x: print(x.result) or test_eq(x.result['height'], 0))  # type: ignore


<Task pending name='Task-10' coro=<ReflectWidget.get_result.<locals>.f() running at /var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_92540/2668587759.py:28>>

In [ ]:
w = BaseExplorerApp(widgets=AD(button=center_button))
test_eq(w.value, {})

Box(children=(ReflectWidget(), BaseExplorerApp(children=(Box(children=(Button(button_style='warning', descript…

In [ ]:
button = W.Button(description='Button', button_style='warning', layout=W.Layout(height='320px', width='100%'))
w = BaseExplorerApp(widgets=AD(button=button), height='120px', layout=AD(max_height='220px'))
test_eq(w.value, {})

Box(children=(ReflectWidget(), BaseExplorerApp(children=(Box(children=(Button(button_style='warning', descript…

In [ ]:
class TestApp(BaseExplorerApp):
    def setup_widgets(self, widgets):
        return AD(text = W.Textarea(layout=W.Layout(width='100%', height='100%')))

Note `TestApp` is equivalent to `BaseExplorerApp(widgets=AD(text=WW.Textarea(layout=W.Layout(width='100%', height='100%')))`

In [ ]:
w = TestApp(text='hello world', height='40px', width='100%')
test_eq(w.value, {'text': 'hello world'})


Box(children=(ReflectWidget(), TestApp(children=(Box(children=(Textarea(value='', layout=Layout(height='100%',…

`TestApp` value is read-only. But being a `Dict`, can be modified in place but with no effect whasoever on the component widgets.  
To modify the app value, you must modify interactively o programatically the widgets.

In [ ]:
with test_raises(T.TraitError):
    w.value = {'text': 'bye, bye'}

In [ ]:
w.widgets.text.value = 'bye, bye'  # type: ignore
test_eq(w.value, {'text': 'bye, bye'})

## Debug view

In [ ]:
# %%html
# <style>
#     .widget-radio-box {flex-direction: row !important;}
#     .widget-radio-box label{margin:5px !important; width: auto !important;}
# </style>


In [ ]:
#| export
class DebugWidget(BaseExplorerApp):
    def clear(self, *args):
        self._w.debug_view.value = ''
    def log(self, msg):
        self._w.debug_view.value = f"{msg}\n{self._w.debug_view.value}"
    def log_inc(self, msg=''):
        first, sep, rest = self._w.debug_view.value.partition('\n')  # type: ignore
        first += '.' + msg
        self._w.debug_view.value = f"{first}\n{rest}"

    # ---------- ---------- UX ---------- ----------
    def setup_ux(self, state):
        super().setup_ux(state)
        self._w.clear_btn.on_click(self.clear)

    def __init__(self, text: str = '', **kwargs):
        kwargs.pop('dcons', None)
        super().__init__(
            debug_view=text, 
            dcons=None, 
            reflect=False, 
            widgets = AD(
                clear_btn = W.Button(
                    description='clear', layout={'position':'absolute','width':'2em'}), 
                debug_view = W.Textarea(
                    value='', layout=W.Layout(width='99.6%', margin='0px', height="100%")), 
            ),
            height = '96px', 
            **kwargs)

@F.patch
def _setup_dcons(self: BaseExplorerApp, dcons: bool | DebugWidgetP | None = None):
    _dcons = NOOPDCONS
    if dcons is True:
        _dcons = DebugWidget(show=False)
    elif isinstance(dcons, DebugWidget):
        _dcons = dcons
    assert isinstance(_dcons, DebugWidget | _NoopDebugWidget)
    _dcons.clear()
    return _dcons


In [ ]:
txt = '''asfasd fdf
fgsdgfsdfg dfghdfhg
sadfsdf {2334} 345456
wqewer asasdf {} []'
sadflsdfd
qopeim, njkhas'''
dcons = DebugWidget(txt)

Box(children=(DebugWidget(children=(Box(children=(Button(description='clear', layout=Layout(width='2em'), styl…

In [ ]:
test_eq(dcons.value, {'debug_view': txt})
test_eq(dcons._w.debug_view.value, txt)


In [ ]:
dcons.clear()
test_eq(dcons.value, {'debug_view': ''})
test_eq(dcons._w.debug_view.value, '')


## Select Detail App

In [ ]:
#| export
class SelectDetailApp(BaseExplorerApp):
    _css = """
        .widget-select select, .widget-output .jp-OutputArea {
        background-color: transparent;
        color: var(--vscode-editor-foreground);
    }
    """
    # ---------- ---------- display ---------- ----------
    def d(self):
        out = self.detail
        # print(self.value)
        if isinstance(out, W.Output):
            v: dict = cast(dict, self.value)
            try:
                out.append_stdout(f"{v['select']}\n")
            except KeyError:
                pass
        
    # ---------- ---------- UI view ---------- ----------
    def setup_layout(self):
        center = W.Box(
                [self.detail],
                layout=W.Layout(
                    height='100%', width='100%', margin='0px', border='0.1px solid red')
            )
        return dict(left_sidebar=self.select, center=center)
    # ---------- ---------- UX ---------- ----------
    def on_value_change(self, change):
        out = self.detail
        # print(f"{time.perf_counter()} - {type(self).__name__}.on_value_changed - {change = }")
        if isinstance(out, W.Output):
            out.clear_output()  # type: ignore
        loop = asyncio.get_event_loop()
        loop.call_later(0.1, self.d)
        # self.d()
    
    # def setup_ux(self, state):
    #     super().setup_ux(state)

    def __init__(self, 
            select: W.DOMWidget, 
            detail: W.DOMWidget | None = None, 
            height: int = 220,
            pane_widths: list[int|str] = [2, 4, 3],
            pane_heights: list[int|str] = [1, 5, 1],
            **kwargs
    ):
        self.detail = W.Output() if detail is None else detail
        self.select = select
        super().__init__(
            widgets = AD(select=self.select, detail=self.detail),
            height = f"{height}px",
            pane_widths = pane_widths,
            pane_heights = pane_heights,
            **kwargs
        )

In [ ]:
left = create_expanded_button('Left', 'info')
center = create_expanded_button('Center', 'warning')
center.layout.width = '100%'  # type: ignore

app = SelectDetailApp(left, center)
test_eq(app.value, {})

Box(children=(SelectDetailApp_CSS(), ReflectWidget(), SelectDetailApp(children=(Button(button_style='info', de…

In [ ]:
options=['Linux', 'Windows', 'macOS', 'Other', 'Other2', '00_imports.ipynb', 'Linux', 
            'Windows', 'macOS', 'Other', 'Other2', '00_imports.ipynb', 'Other', 'Other2', '00_imports.ipynb']

sel = W.Select(
    options=options,
    value=None,
    # layout=W.Layout(width='100%', height='100%', margin='0px', border='0px solid black'),
    layout=W.Layout(width='100%', height='100%', margin='0px'),
    # layout=W.Layout(width='100%', height='100%'),
    disabled=False
)

app = SelectDetailApp(sel, pane_widths=['120px', 4, 4], dcons=True)
sel.observe(lambda x: app.dcons.log(str(sel.value)), names='value')  # type: ignore
# app
# test_eq(app.value, {'select': 'macOS'})
test_eq(app.value, {})


Box(children=(SelectDetailApp_CSS(), ReflectWidget(), SelectDetailApp(children=(Select(layout=Layout(grid_area…

DebugWidget(children=(Box(children=(Button(description='clear', layout=Layout(width='2em'), style=ButtonStyle(…

In [ ]:
app.select.value = 'Windows'
test_eq(app.value, {'select': 'Windows'})


In [ ]:
#| export
class ListSelect(SelectDetailApp):
    def __init__(self, options, value, **kwargs):
        sel = W.Select(
            options=options,
            layout=W.Layout(width='100%', height='100%', margin='0px', border='0px solid red'),
            disabled=False
        )
        super().__init__(sel, 
                         pane_widths=kwargs.pop('pane_widths', ['120px', 4, 4]), **kwargs)
        sel.value = value

In [ ]:
app = ListSelect(options, value='macOS')
test_eq(app.value, {'select': 'macOS'})

Box(children=(SelectDetailApp_CSS(), ReflectWidget(), ListSelect(children=(Select(layout=Layout(border_bottom=…

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.expand_frame_repr', True)
# pd.set_option("display.width", 2000)
pd.set_option("display.max_columns", None) # show all cols
pd.set_option('display.max_colwidth', None) # show full width of showing cols
pd.set_option("display.expand_frame_repr", False) # print cols side by side as it's supposed to be

In [ ]:
df = pd.DataFrame({
    'int': [1, 2, 3, ]*5,
    'float': [3.14, 6.28, 9.42, ]*5,
    'str': ['A', 'B', 'C', ]*5,
    'bool': [True, False, True, ]*5,
    'date': [dt.date(2019, 1, 1), dt.date(2020, 1, 1), dt.date(2020, 1, 10), ]*5,
    'datetime': [dt.datetime(2019, 1, 1, 10), dt.datetime(2020, 1, 1, 12), dt.datetime(2020, 1, 10, 13), ]*5,
    '1int': [1, 2, 3, ]*5,
    '1float': [3.14, 6.28, 9.42, ]*5,
    '1str': ['A', 'B', 'C', ]*5,
    '1bool': [True, False, True, ]*5,
    '1date': [dt.date(2019, 1, 1), dt.date(2020, 1, 1), dt.date(2020, 1, 10), ]*5,
    '1datetime': [dt.datetime(2019, 1, 1, 10), dt.datetime(2020, 1, 1, 12), dt.datetime(2020, 1, 10, 13), ]*5,
}, index=[1, 2, 3, ]*5)


In [ ]:
detail: W.Output = cast(W.Output, app.detail)
detail.clear_output()


In [ ]:
detail.append_display_data(df.style)


## Colophon
----

In [ ]:
import fastcore.all as F
import shutil
if F.IN_NOTEBOOK:
    import nbdev; nbdev.nbdev_export('01_base_widget.ipynb')
shutil.copyfile('src/reflect.js', '../vwidget/static/reflect.js');